<a href="https://colab.research.google.com/github/jhejna/rlworkshop/blob/main/MB_RL_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup

In [ ]:
%%bash
# install required system dependencies
apt-get install -y xvfb x11-utils
# install required python dependencies (might need to install additional gym extras depending)
pip install gym pyvirtualdisplay

In [ ]:
import pyvirtualdisplay
_display = pyvirtualdisplay.Display(visible=False, size=(640, 480))
_ = _display.start()

In [ ]:
!echo $DISPLAY

In [ ]:
# Test rendering a gym environment
import gym
import matplotlib.pyplot as plt
from IPython import display

env = gym.make('CartPole-v1')
num_ep = 0
obs = env.reset()
img = plt.imshow(env.render(mode='rgb_array'))
for _ in range(100):
    ac = env.action_space.sample()
    obs, reward, done, info = env.step(ac)
    img.set_data(env.render(mode='rgb_array')) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    if done:
        obs = env.reset()
        num_ep += 1
env.close()
del env
del img
print("Episodes:", num_ep)

# 1. PETS
For this workshop we will implement the Probablistic Ensembles with Trajectory Sampling (PETS) algorithm from "Deep Reinforcement Learning in a Handful of Trials Using Probabilistic Models"

Here is a link to the paper: https://arxiv.org/abs/1805.12114

The basic idea is this: learned dynamics models often accumulate a large amount of error that leads to bad plans. By planning through an ensemble, or collection of models, we can mitigate this error.

Similar to the last workshop, we will work with the simple discrete cartpole environment.


In [ ]:
# Import the required modules
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import gym

## a) Dynamics Models
PETS uses an ensemble of probablistic models. What does this mean?
1. Rather than just predicting the next state, we predict a distribution over the possible next states. In the authors implementation, this takes the form of a diagonal multivariate gaussian distribution.
2. Rather than having a single dynamics model and sampling from it during planning, we use an ensemble to reduce error.

In this section, you will implement both a guassian dynamics model in the form of an MLP and an ensemble wrapper.

Here are a few things to remember when making the dynamics model:
1. The dynamics model takes in the state and action and predicts the next state. 
2. For the guassian model, we need to predict a mean and variance for each component of the state.
3. Dynamics models generally perform better when they predict the delta to the next state. For example `next_state = prediction + state` 

In [ ]:
class GaussMLP(nn.Module):

    def __init__(self, in_dim, out_dim, hidden_dims=[256, 256]):
        super().__init__()
        layers = []
        #YOUR CODE HERE
        #HINT: out_dim is the number of dimensions of the state space
        
        #This defines bounds for the log of the variance
        self.max_logvar = nn.Parameter(torch.ones(1, out_dim, dtype=torch.float32) / 2.0)
        self.min_logvar = nn.Parameter(- torch.ones(1, out_dim, dtype=torch.float32) * 10.0)

    def forward(self, x):
        #YOUR CODE HERE

        # Note, authors use softplus here and bound the variance. We have provided this for you.
        logvar = self.max_logvar - F.softplus(self.max_logvar - logvar)
        logvar = self.min_logvar + F.softplus(logvar - self.min_logvar)
        return mu, logvar
    
class GaussEnsemble(nn.Module):

    def __init__(self, in_dim, out_dim, ensemble_size, hidden_dims=[256, 256]):
        #YOUR CODE HERE
        #HINT: the ensemble is just a collection of Gaussian MLPs

    def forward(self, x):
        means, logvars = [], []
        
        #YOUR CODE HERE
        #HINT: return a list of the mu's and logvar's for each model in the ensemble

        return means, logvars

    

## b) Environment Setup
In model based reinforcement learning, we plan using the reward function. For example, in random shooting, trajectory optimization we will later implement, you rank different action sequences by the sum of their rewards. This means that we will need access to the reward function. By default, this isnt provided in gym, so I have provided it here.

In [ ]:
def reward_function(state):
    x = state[:, 0]
    theta = state[:, 2]
    theta_thres = 12 * 2 * math.pi / 360

    x_done = np.logical_or(x < -2.4, x > 2.4)
    theta_done = np.logical_or(theta < -theta_thres, theta > theta_thres)

    done = np.logical_or(x_done, theta_done)
    return 1 - done

Next, we need to collect data from the environment. In model based RL it's nice to package this in a single function instead of having it in your training loop. We haven't talked about how you actually plan in the environment yet, so we'll assume that you have some `act_fn` that when provided an observation will predict the next action you take. For example `action = act_fn(observation)`

The `collect_data` function will take in an environment, an `act_fn`, and a number of `timesteps`, and use the `act_fn` to collect data for the specified number of timesteps. 

You should be able to figure out how to do this from the model free RL workshop training loop or testing functions. It's nice to log values here so you can see how your policy does.

In [ ]:
def collect_data(env, act_fn, timesteps):
    inputs = np.zeros((timesteps, 6))
    targets = np.zeros((timesteps, 4))

    obs = env.reset()
    ep_rewards = [] #list of episode reward per episode
    ep_reward = 0 
    
    #YOUR CODE HERE
    #HINT: the "done" value returned by env.step() tells you how to split episodes
    #What data do we need to train? What does the model take as input, and what does it try to predict? (ignore the variances for now)

    if len(ep_rewards) == 0:
        print("Total Ep Reward", ep_reward)
    else:
        print("Last Ep Reward", ep_rewards[-1])
    return inputs, targets

## c) MPC Planning with Random Shooting
MPC, or model predictive control, is a way of using dynamics models and a reward function to predict the action that should be taken in an environment. We will implement random shooting in particular.

In random shooting, a number of random trajectories are sampled. Each trajectory is then assigned a total reward using the reward function. Finally, the tracjectory with the highest reward is selected and the first action is used.

Alone, random shooting, well sucks. PETS utilizes the ensemble of probablistic models to make it not so bad. In the paper, they use a more complicated algorithm called the cross entropy method that we will not get into here :).

In this section, implement random shooting for PETS. This will require the following modifications in addition to regular random shooting:
1. Randomly sampling next states using the probablity distribution output by a given model.
2. Compute rewards for each trajectory under all of the different models. Average the final reward across all dynamics models in the ensemble to get a better estimate.

Keep in mind that the slowest part of training our model is now planning. It's crucial that your implementation of random shooting predicts next states for all samples in parallel (ie one large batch). Otherwise, you will be waiting years for everything to finish.

Warning: this is by far the hardest part of the PETS algorithm :)

In [ ]:
def act(obs, models, reward_fn, horizon):
    # Generate plans of length "Horizon"
    # Each model predicts a cost over acting for horizon timesteps.
    
    #YOUR CODE HERE
    #Sample a bunch of random trajectories and return the first action of the one that has the highest predicted reward
    #HINT: The action that is input into the model is a 2-dimensional one-hot vector (i.e. [1,0] or [0,1])
    
    #Return the ordinal encoding of the action (the index of the one in the one-hot vector, 
    #e.g. if the selected action is [1,0] then return 0, if it's [0,1] return 1)
    return best_ac

## d) Training Loop
Implement the training loop for the algorithm. This should be done in two phases:
1. Collect data
2. Train the models

Usually, models are initiallized using a few random episodes and warmed up before trying to plan.

Fill in the loss. Look at the PETS paper to determine what the loss should be. Hint: It should have something for both the mean and variance. Its fine if the loss becomes negative due to the variance term.

In [ ]:
import math

env = gym.make("CartPole-v1")
# State Dim is 2, action is one hot of size 2
batch_size = 64
epochs = 2
iterations = 15
timesteps = 400
horizon = 10

models = [GaussMLP(4+2, 4) for i in range(5)]
parameters = []
for model in models:
    parameters.extend(model.parameters())
optim = torch.optim.Adam(parameters, lr=0.001)

# Collect initial data and train.
inputs, targets = collect_data(env, lambda x: env.action_space.sample(), timesteps)

for i in range(iterations):
    
    for _ in range(epochs):
        perm = np.random.permutation(len(inputs))
        num_full_batches = len(perm) // batch_size
        for i in range(num_full_batches + 1):
            if i != num_full_batches:
                inds = perm[i*batch_size:(i+1)*batch_size]
            else:
                inds = perm[i*batch_size:]
            x = torch.from_numpy(inputs[inds]).float()
            y = torch.from_numpy(targets[inds]).float()

            loss = 0.0
            for model in models:
                mean, logvar = model(x)
                loss += #YOUR CODE HERE
            
            optim.zero_grad()
            loss.backward()
            optim.step()
        print("Loss", loss.item())
    # Collect new data
    new_inputs, new_targets = collect_data(env, lambda obs: act(obs, models, reward_function, horizon), timesteps)
    inputs = np.concatenate((inputs, new_inputs), axis=0)
    targets = np.concatenate((targets, new_targets), axis=0)

## e) Tuning
Play with the hyper-parameters and see how high you can get the reward!

Model based RL is _even_ more unstable than model free RL :). Good luck!

Within a around 75 episodes I was able to get a reward of over 350, though my implementation was pretty unstable and tended to crash :P. This is the problem with using random shooting! My loss nanned a few times, this can be fixed by better controlling the variance term of the loss.

# 2. VAEs and World Models

In this section you'll implement a very simple VAE that could be used in some type of world model for MB RL from images. Though typically you would do this by collecting data from environments, we will do this on the MNIST dataset for simplicity.

Here is the link to the original VAE Paper:
https://arxiv.org/pdf/1312.6114.pdf

And the world models paper: https://arxiv.org/abs/1803.10122


## a) Data Collection

Note sometimes this tends to error because of the display environment being buggy in colab. If it crashes, go ahead and re-run the set display block at the top of the noteobok.

In [ ]:
import gym
import numpy as np
from PIL import Image, ImageOps

env = gym.make('CartPole-v1')
data = []
data_size = 2000

obs = env.reset()
ac = 0
for _ in range(data_size):
    if np.random.rand() < 0.5:
        ac = env.action_space.sample()
    obs, reward, done, info = env.step(ac)
    img = Image.fromarray(env.render(mode='rgb_array'))
    img_gray = ImageOps.grayscale(img)
    img_resize = img_gray.resize((56,56))
    img_crop = img_resize.crop((8,8, 48, 48))
    data.append(np.array(img_crop))
    if done:
        obs = env.reset()
env.close()
del env
del img


In [ ]:
# Now Shuffle the data, and visualize some images
data = np.array(data)
np.random.shuffle(data)

# Visualize some random images
from matplotlib import pyplot as plt
plt.imshow(data[0], cmap='gray')
plt.show()
plt.imshow(data[1], cmap='gray')
plt.show()
plt.imshow(data[2], cmap='gray')
plt.show()

In [ ]:
# Normalize data between -1 and 1
data = data / 255.0 # Between zero and one
data *= 2 # Between zero and two
data -= 1 # Between -1 and 1

# Expand dims of data and reorder in order to fit pytorch image conventions N x C x W x H
data = np.expand_dims(data, 1)
print("Data Shape", data.shape)

np.save('data.npy', data)

# Test loading the data
data = np.load('data.npy')

## b) Model Architectures

Next, you need to define a simple convolutional architecture for the encoder and decoder. Here are things to consider:

1. Choose an appropriate size for the latent z
2. Remember that the encoder must predict both a mean and standard deviation.
3. The standard deviation (or variance) must be a positive quantity!
4. The encoder and decoder architectures should closely mirror eachother.


In [ ]:
import torch
import numpy as np
from torch import nn

class Encoder(nn.Module):

    def __init__(self, input_channels, latent_dim):
        super().__init__()
        #YOUR CODE HERE

    def forward(self, x):
        #YOUR CODE HERE
        return mu, logvar

class Decoder(nn.Module):

    def __init__(self, output_channels, latent_dim):
        super().__init__()
        #YOUR CODE HERE
        #HINT: Add a tanh at the end to bound inputs/outpus

    def forward(self, x):
        #YOUR CODE HERE

class VAE(nn.Module):

    def __init__(self, channels, latent_dim):
        super().__init__()
        #YOUR CODE HERE
        #HINT: It's just an encoder + a decoder

    def forward(self, x):
        #YOUR CODE HERE
        #Pass the input through the encoder then decoder, and return the loss
        #HINT: the loss should have MSE (Mean Squared Error) and KLD (KL Divergence) components
        
        return loss

## c) Training Loop
Now we code the training loop. We can do this really easily with pytorch data loaders!

In [ ]:
batch_size = 128
lr = 1e-3
latent_dim = 16
epochs = 50

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# create data loader
data = np.load('data.npy')
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

model = VAE(1, latent_dim)

model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    for batch in trainloader:
        batch = batch.to(device).float()
        optimizer.zero_grad()
        loss = model(batch)
        loss.backward()
        optimizer.step()

    print("Finished epoch", epoch, "loss", loss.item())

# Now let's generate some samples

## d) Evaluation

Now assess the quality of the learned model by sampling from the latent (sampling z, then examining x_hat)

Note that an easy way of displaying the images is torchvision makegrid, it does a lot of the deprocessing work for you! 


It won't be super good, but you should be able to see the base of the cart! If you mess with the architecture I'm sure you can get it to peform better!

In [ ]:
import torchvision
from matplotlib import pyplot as plt
from PIL import Image

# Samples!
num_samples = 24
samples = model.decoder(torch.randn(num_samples, latent_dim).to(device)).detach().cpu()
sample_grid = torchvision.utils.make_grid(samples)
sample_img = torchvision.utils.save_image(sample_grid, 'samples.png')
plt.imshow(Image.open('samples.png'))
plt.show()
